In [5]:
import ollama

def generate_summary(keyword,data,region):
    data_json = data.to_json()
    prompt = f"Explain why the keyword '{keyword}' is trending in '{region}' based on the following data:\n\n{data_json}\n\nInclude possible reasons and relevant context.Make it point form and make it clean to visible"
    response = ollama.chat(
        model= "llama3",
        messages=[{
            "role": "user",
            "content": prompt
        }]
    )
    return str(response['message']['content'])


In [10]:
import os
import time
import pandas as pd
import matplotlib.pyplot as plt
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import gradio as gr
from openai import OpenAI
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import os
import time

def download_trends_csv(keyword):
    options = Options()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument("--disable-dev-shm-usage")
    prefs = {"download.default_directory": os.getcwd()}
    options.add_experimental_option("prefs", prefs)
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    try:
        driver.get('https://trends.google.com/trends/')
        time.sleep(2)
        
        # Search for the keyword
        search_box = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'input.Fgl6fe-fmcmS-wGMbrd.fnqhWc'))
        )
        search_box.clear()
        search_box.send_keys(keyword)
        search_box.send_keys(Keys.RETURN)
        time.sleep(2)
        
        selected_text_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.hierarchy-select"))
        )
        selected_text = selected_text_element.find_element(By.CSS_SELECTOR, "span").text
        
        # Select "Past 5 years"
        print("Selecting 'Past 5 years'...")
        time_picker = driver.find_element(By.CSS_SELECTOR, 'custom-date-picker')
        time_picker.click()
        past_five_years_option = driver.find_element(By.XPATH, "//md-option/div[contains(text(), 'Past 5 years')]")
        past_five_years_option.click()
        time.sleep(2)

        # Check if the line chart component is present
        try:
            line_chart_component = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div.line-chart-body-wrapper'))
            )
        except:
            print("No data for the specified keyword.")
            return None
        
        # Identify and click the download button
        download_buttons = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'button.widget-actions-item.export'))
        )
        if line_chart_component and download_buttons:
            driver.execute_script("arguments[0].scrollIntoView(true);", download_buttons[0])
            driver.execute_script("arguments[0].click();", download_buttons[0])
            time.sleep(5)  # Wait for the file to download

            # Locate the downloaded file
            for file in os.listdir(os.getcwd()):
                if file.endswith('.csv'):
                    return file, selected_text
        else:
            return None
    
    finally:
        driver.quit()

def plot_trend(data, keyword,region):
    xn = data.columns[0]
    yn = data.columns[1]
    x = data[xn]
    y = data[yn]
    plt.figure(figsize=(50, 6))
    plt.plot(x, y)
    plt.title(f'Trend over time for {keyword} in {region}')
    plt.xlabel('Date')
    plt.ylabel('Interest')  # Rotate x-axis labels for better visibility
    plt.legend()
    plt.grid(True)
    plt.show()
    plot_filename = f'{keyword}_in_{region}_trend.png'
    plt.savefig(plot_filename)
    return plot_filename

# def generate_summary(keyword, data):
#     data_json = data.to_json()
#     prompt = f"Explain why the keyword '{keyword}' is trending based on the following data:\n\n{data_json}\n\nInclude possible reasons and relevant context."
#     completion = client.chat.completions.create(
#         model="gpt-3.5-turbo-1106",
#         messages=[
#             {"role": "user", "content": prompt},
#         ]
#     )
#     return completion.choices[0].message['content']

def analyze_trend(keyword):
    try:
        csv_file, selected_text = download_trends_csv(keyword)
        if not csv_file:
            return "No data found for the given keyword.", None

        data = pd.read_csv(csv_file,skiprows=[0])
        if data.empty:
            return "No data found for the given keyword.", None
        
        plot_filename = plot_trend(data, keyword,selected_text)
        summary = generate_summary(keyword, data,selected_text)
        
        os.remove(csv_file)
        
        return summary, plot_filename
    except Exception as e:
        return str(e), None

# Gradio interface
iface = gr.Interface(
    fn=analyze_trend,
    inputs=["text"],
    outputs=["textbox", "image"],
    title="Google Trends Analysis",
    description="Enter a keyword to analyze its trend and get a summary explaining the trend.",
    theme="compact"
)

# Launch the interface
iface.launch()


/home/dinuk/anaconda3/envs/llama/lib/python3.11/site-packages/gradio/blocks.py:983: UserWarning: Cannot load compact. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/compact (Request ID: Root=1-669168ce-5d811818400caab507ab957b;68ab4d2f-cd5f-4b9d-bed9-e3fe1a6e71e5)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


Selecting 'Past 5 years'...


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
/tmp/ipykernel_1569/1623396940.py:102: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [ ]:
import pandas as pd
data = pd.read_csv("/mnt/d/Self_Learning/UpWork/trending/multiTimeline.csv",skiprows=[0])
data.head()
print(data.columns[0])
xn = data.columns[0]
yn = data.columns[1]

In [ ]:
print(yn)

In [ ]:
import matplotlib.pyplot as plt
x = data[xn]
y = data[yn]
print(x)
print(y)
plt.figure(figsize=(50, 6))
plt.plot(x, y)
plt.title(f'Trend over time for Cricket')
plt.xlabel('Date')
plt.ylabel('Interest')  # Rotate x-axis labels for better visibility
plt.legend()
plt.grid(True)
plt.show()